Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

In [4]:
initial_load_user_group = ''
delta_load_user_group = ''

In [5]:
import ast

# convert string parameter to list of strings
delta_load_user_group= ast.literal_eval(delta_load_user_group)
initial_load_user_group = ast.literal_eval(initial_load_user_group)

In [6]:
import requests
import msal
import logging

linked_service = ""
keyvault_name = ""

authority = mssparkutils.credentials.getSecret(linkedService=linked_service,akvName=keyvault_name,secret='')
client_id = mssparkutils.credentials.getSecret(linkedService=linked_service,akvName=keyvault_name,secret='')
secret = mssparkutils.credentials.getSecret(linkedService=linked_service,akvName=keyvault_name,secret='')


scope = ["https://graph.microsoft.com/.default"]
base_endpoint = "https://graph.microsoft.com/v1.0"

app = msal.ConfidentialClientApplication(client_id, authority=authority,client_credential=secret)

result = None
result = app.acquire_token_silent(scope, account=None)

if not result:
    logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
    result = app.acquire_token_for_client(scope)

if "access_token" not in result:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id")) 

headers = {'Authorization': 'Bearer ' + result['access_token']}

def get_group_members(base_endpoint, headers, group_id):
    endpoint = base_endpoint + '/groups/' + group_id + '/members' 
    graph_data = requests.get(endpoint,headers=headers,).json()
    return(graph_data)

def delete_group_members(base_endpoint, headers, group_id,member_ids):
    for member_id in member_ids:
        endpoint = base_endpoint + '/groups/' + group_id + '/members/' + member_id + '/$ref'
        response = requests.delete(endpoint, headers=headers)
    return(response)

def add_group_members(base_endpoint, headers, group_id,members_data):
    endpoint = base_endpoint + '/groups/' + group_id
    response = requests.patch(endpoint,json=members_data, headers=headers)
    return(response)


In [7]:
from pyspark.sql.functions import lit, col

df_members = get_group_members(base_endpoint,headers,initial_load_user_group[0])

member_ids = []
for member_value in df_members['value']:
    member_ids.append(member_value['id'])

members_data = []
for member_id in member_ids:
    members_data.append('https://graph.microsoft.com/v1.0/directoryObjects/' + member_id)

members_data = {
    "members@odata.bind": members_data
}

df_response = add_group_members(base_endpoint,headers,delta_load_user_group[0],members_data)
df_response

if len(df_members['value']) > 0:
    delete_group_members(base_endpoint,headers,initial_load_user_group[0],member_ids)

# update the inital load completion flag in Cosmos DB 
df = spark.read.format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "CosmosDB")\
            .option("spark.cosmos.container", "users")\
            .load()

# Uncomment this if your Linked Service is enabled with a private endpoint 
#df = spark.read.format("cosmos.oltp")\
#            .option("spark.cosmos.useGatewayMode", True)\
#            .option("spark.synapse.linkedService", "CosmosDB")\
#            .option("spark.cosmos.container", "users")\
#            .load()

df = df.filter(col('user_id').isin(member_ids))
df = df.withColumn('initialload_completion_flag', lit(1))
member_ids = df.select("user_id").rdd.flatMap(lambda x: x).collect()

# append updates the existing record with flag from 0 to 1
df.write.format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "CosmosDB")\
    .option("spark.cosmos.container", "users")\
    .mode('append')\
    .save()

# Uncomment this if your Linked Service is enabled with a private endpoint 
#df.write.format("cosmos.oltp")\
#    .option("spark.cosmos.useGatewayMode", True)\
#    .option("spark.synapse.linkedService", "CosmosDB")\
#    .option("spark.cosmos.container", "users")\
#    .mode('append')\
#    .save()